# This notebook retreives all the subjects that were ever prescribed at least one of the following opioids

Buprenorphine
Codeine
Dihydrocodeine
Fentanyl
Hydrocodone
Hydromorphone
Levorphanol
Meperidine
6-Acetylmorphine
Morphine-3-Glucuronide
---- both ---- Nalorphine
-------------- Naloxone
Oxycodone
Oxymorphone
----------- acetaminophen --------------- 
------------ methadone

### Definite Narcotics:
Oxymorphone
Oxycodone
Morphine
Meperidine
Hydromorphone
Hydrocodone
Fentanyl
Codeine  (& Dihydrocodeine)
Buprenorphine
Levorphanol  ---- Not high risk
Methadone  ---- Not high risk
carfentanil --- synthetic analegsic
heroin

### Anti narcotics:
methadone
Naloxone
Nalorphine

In [1]:
import numpy as np
import pandas as pd
import operator
import pickle

In [2]:
## Reading MIMIC Data
data_dir = "../Data/"
drug_codes = pd.read_csv(data_dir+"DRGCODES.csv.gz", compression="gzip")
patients = pd.read_csv(data_dir+"PATIENTS.csv.gz", compression="gzip")

In [3]:
prescriptions = pd.read_csv(data_dir+"PRESCRIPTIONS.csv.gz", compression="gzip")
prescriptions.head(5)

/share/apps/python3/3.5.3/intel/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,DRUG_NAME_POE,DRUG_NAME_GENERIC,FORMULARY_DRUG_CD,GSN,NDC,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE
0,2214776,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Tacrolimus,Tacrolimus,Tacrolimus,TACR1,021796,469061711.0,1mg Capsule,2,mg,2,CAP,PO
1,2214775,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Warfarin,Warfarin,Warfarin,WARF5,006562,56017275.0,5mg Tablet,5,mg,1,TAB,PO
2,2215524,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Heparin Sodium,NaN,NaN,HEPAPREMIX,006522,338055002.0,"25,000 unit Premix Bag","25,000",UNIT,1,BAG,IV
3,2216265,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,BASE,D5W,NaN,NaN,HEPBASE,NaN,0.0,HEPARIN BASE,250,ml,250,ml,IV
4,2214773,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Furosemide,Furosemide,Furosemide,FURO20,008208,54829725.0,20mg Tablet,20,mg,1,TAB,PO


In [4]:
narcotic_drugs = ['oxymorphone', 'oxycodone', 'morphine', 'meperidine', 
                  'hydromorphone', 'hydrocodone', 'fentanyl', 'codeine', 'buprenorphine', 'levorphanol', 'methadone']
anti_narcotic_drugs = ['methadone', 'naloxone', 'nalorphine']
both_drugs = narcotic_drugs + anti_narcotic_drugs

In [7]:
heroin = prescriptions[prescriptions["DRUG"].str.contains('heroin')]
len(carfentanil_drug)

0

In [5]:
## Convert drug names to lower case

prescriptions["DRUG"] = prescriptions["DRUG"].str.lower()

In [31]:
prescriptions_with_narcotic_drugs = prescriptions[prescriptions["DRUG"].str.contains("|".join(narcotic_drugs))]
prescriptions_with_anti_narcotic_drugs = prescriptions[prescriptions["DRUG"].str.contains("|".join(anti_narcotic_drugs))]
prescriptions_with_both_drugs = prescriptions[prescriptions["DRUG"].str.contains("|".join(both_drugs))]

In [30]:
## Mixed narcotic and anti-narcotic drug - methadone
prescriptions_with_mixed_drugs = prescriptions[prescriptions["DRUG"].str.contains("methadone")]

In [36]:
## Drop buprenorphine-naloxone (8mg-2mg) frm ant-narcotics

prescriptions_with_anti_narcotic_drugs = prescriptions_with_anti_narcotic_drugs[prescriptions_with_anti_narcotic_drugs["DRUG"] != ('buprenorphine-naloxone (8mg-2mg)')]

In [37]:
narcotic_in_prescription = set(prescriptions_with_narcotic_drugs["DRUG"])
anti_narcotic_in_prescription = set(prescriptions_with_anti_narcotic_drugs["DRUG"])
mixed_drug_in_prescription = set(prescriptions_with_mixed_drugs["DRUG"])

In [38]:
print("Narcotic Drugs: %i" %(len(narcotic_in_prescription)))
print(narcotic_in_prescription)
print("Anti-Narcotic Drugs: %i" %(len(anti_narcotic_in_prescription)))
print(anti_narcotic_in_prescription)
print("Mixed Drugs: %i" %(len(mixed_drug_in_prescription)))
print(mixed_drug_in_prescription)

Narcotic Drugs: 89
{'hydromorphone hcl', 'oxycodone (immediate release) ', 'neo*po*morphine', 'methadone hcl', 'methadone (dispersible tablet)', 'morphine sulfate (oral soln.)', 'morphine sulfate (0.5-4mg)', 'morphine (pf)', 'morphine concentrate', 'morphine sulfate (pf)', 'tylenol-codeine #3', 'oxycodone liquid', 'hydrocodone-acetaminophen (5mg-500mg)', 'fentanyl ', 'hydromorphone solution', 'hydromorphone p.f.', 'morphine sulfate', 'oxycodone-acetaminophen elixir', 'morphine sulfate for inhalation', 'codeine phosphate', 'morphine su', 'morphine sulfate (concentrated oral soln)', 'morphine sulfate ir', 'chlorpheniramine-hydrocodone', 'meperidine pf', 'morphine sr (ms contin)', 'methadone', 'morphine sulfate pf', 'oxycodone ', 'hydromorphone', 'codeine-guaifenesin', 'morphine sulfate (4-8mg)', 'morphine sulfate (1-5mg)', 'buprenorphine hcl', 'hydromorphone-hp', 'acetaminophen w/codeine', 'morphine sulfate (1-10mg)', 'oxycodone hcl', 'codeine sulfate', 'morphine sulfate (2-5mg)', 'oxyco

In [40]:
print("prescriptions_with_narcotic_drugs: %i" %(len(prescriptions_with_narcotic_drugs)))
print("prescriptions_with_anti_narcotic_drugs: %i" %(len(prescriptions_with_anti_narcotic_drugs)))
print("prescriptions_with_mixed_drugs (methadone): %i" %(len(prescriptions_with_mixed_drugs)))
print("prescriptions_with_both_drugs: %i" %(len(prescriptions_with_both_drugs)))

prescriptions_with_narcotic_drugs: 216947
prescriptions_with_anti_narcotic_drugs: 5672
prescriptions_with_mixed_drugs (methadone): 4398
prescriptions_with_both_drugs: 218221


In [57]:
with open('../Data/Opioids/Prescriptions/prescriptions_with_narcotic_drugs.pickle', 'wb') as pickler:
    pickle.dump(prescriptions_with_narcotic_drugs, pickler, protocol=pickle.HIGHEST_PROTOCOL)
with open('../Data/Opioids/Prescriptions/prescriptions_with_anti_narcotic_drugs.pickle', 'wb') as pickler:
    pickle.dump(prescriptions_with_anti_narcotic_drugs, pickler, protocol=pickle.HIGHEST_PROTOCOL)
with open('../Data/Opioids/Prescriptions/prescriptions_with_mixed_drugs.pickle', 'wb') as pickler:
    pickle.dump(prescriptions_with_mixed_drugs, pickler, protocol=pickle.HIGHEST_PROTOCOL)
with open('../Data/Opioids/Prescriptions/prescriptions_with_both_drugs.pickle', 'wb') as pickler:
    pickle.dump(prescriptions_with_both_drugs, pickler, protocol=pickle.HIGHEST_PROTOCOL)


In [50]:
narcotic_drug_count = {'oxymorphone':0, 'oxycodone':0, 'morphine':0, 'meperidine':0, 
                  'hydromorphone':0, 'hydrocodone':0, 'fentanyl':0, 'codeine':0, 'buprenorphine':0, 'levorphanol':0, 'methadone':0}
for index, row in prescriptions_with_narcotic_drugs.iterrows():
    if ('oxymorphone' in row["DRUG"]):
        narcotic_drug_count['oxymorphone'] += 1
    elif 'oxycodone' in row["DRUG"]:
        narcotic_drug_count['oxycodone'] += 1
    elif 'morphine' in row["DRUG"]:
        narcotic_drug_count['morphine'] += 1
    elif 'meperidine' in row["DRUG"]:
        narcotic_drug_count['meperidine'] += 1
    elif 'hydromorphone' in row["DRUG"]:
        narcotic_drug_count['hydromorphone'] += 1
    elif 'hydrocodone' in row["DRUG"]:
        narcotic_drug_count['hydrocodone'] += 1
    elif 'fentanyl' in row["DRUG"]:
        narcotic_drug_count['fentanyl'] += 1
    elif 'codeine' in row["DRUG"]:
        narcotic_drug_count['codeine'] += 1
    elif 'buprenorphine' in row["DRUG"]:
        narcotic_drug_count['buprenorphine'] += 1
    elif 'levorphanol' in row["DRUG"]:
        narcotic_drug_count['levorphanol'] += 1
    elif 'methadone' in row["DRUG"]:
        narcotic_drug_count['methadone'] += 1

print(narcotic_drug_count)

{'buprenorphine': 50, 'hydromorphone': 53202, 'codeine': 2356, 'methadone': 4398, 'fentanyl': 35734, 'oxymorphone': 4, 'levorphanol': 0, 'oxycodone': 44538, 'hydrocodone': 1998, 'morphine': 68045, 'meperidine': 6622}


In [51]:
anti_narcotic_drug_count = {'methadone':0, 'naloxone':0, 'nalorphine':0}
for index, row in prescriptions_with_anti_narcotic_drugs.iterrows():
    if ('methadone' in row["DRUG"]):
        anti_narcotic_drug_count['methadone'] += 1
    elif 'naloxone' in row["DRUG"]:
        anti_narcotic_drug_count['naloxone'] += 1
    elif 'nalorphine' in row["DRUG"]:
        anti_narcotic_drug_count['nalorphine'] += 1
        
print(anti_narcotic_drug_count)

{'naloxone': 1274, 'nalorphine': 0, 'methadone': 4398}


A prescription can be repeated for same subject. Hence, we look at the number of unique subjects that narcotics/anti-narcotics were prescribed.

In [53]:
subjects_with_narcotic_drugs = set(prescriptions_with_narcotic_drugs["SUBJECT_ID"])
subjects_with_anti_narcotic_drugs = set(prescriptions_with_anti_narcotic_drugs["SUBJECT_ID"])
subjects_with_mixed_drugs = set(prescriptions_with_mixed_drugs["SUBJECT_ID"])
subjects_with_both_drugs = set(prescriptions_with_both_drugs["SUBJECT_ID"])

In [54]:
print("subjects_with_narcotic_drugs: %i" %(len(subjects_with_narcotic_drugs)))
print("subjects_with_anti_narcotic_drugs: %i" %(len(subjects_with_anti_narcotic_drugs)))
print("subjects_with_mixed_drugs (methadone): %i" %(len(subjects_with_mixed_drugs)))
print("subjects_with_both_drugs: %i" %(len(subjects_with_both_drugs)))

subjects_with_narcotic_drugs: 29959
subjects_with_anti_narcotic_drugs: 1437
subjects_with_mixed_drugs (methadone): 736
subjects_with_both_drugs: 29991


In [56]:
subjects_with_narcotic_and_anti_narcotic = subjects_with_narcotic_drugs.intersection(subjects_with_anti_narcotic_drugs)
len(subjects_with_narcotic_and_anti_narcotic)

1405